In [8]:
import numpy as np

### Board for UCS solution

In [9]:
class UCSBoard:
    def __init__(self, board = None, empty_tile = -1, path_cost = -1) -> None:
        self.board = board if board is not None else np.array([])
        self.empty_tile = empty_tile
        self.path_cost = path_cost
    def __str__(self) -> str:
        return str(self.board)
    def get_board(self):
        return self.board
    def get_empty_tile(self):
        return self.empty_tile
    def get_path_cost(self):
        return self.path_cost
    def __eq__(self, other):
        return (self.board == other.board).all()
    def __gt__(self, other):
        return self.path_cost > other.path_cost
    def __lt__(self, other):
        return self.path_cost < other.path_cost
    def __eq__(self, __value: object) -> bool:
        if isinstance(__value, UCSBoard):
            return np.array_equal(self.board, __value.board)
        return False
    def __hash__(self) -> int:
        return hash(tuple(self.board.flatten()))

### Board for AStar solution

In [10]:
class AStarBoard(UCSBoard):
    def __init__(self, board=None, empty_tile=-1, path_cost=-1, inversion_count_horizontal=-1, inversion_count_vertical=-1, heuristic = -1) -> None:
        super().__init__(board, empty_tile, path_cost)
        self.inversion_count_horizontal = inversion_count_horizontal
        self.inversion_count_vertical = inversion_count_vertical
        self.heuristic = heuristic

    def get_heuristic(self):
        return self.heuristic

    def get_total_cost(self):
        return self.heuristic + self.path_cost

    def __eq__(self, other):
        return (self.board == other.board).all()

    def __gt__(self, other):
        return self.get_total_cost() > other.get_total_cost()

    def __lt__(self, other):
        return self.get_total_cost() < other.get_total_cost()
    
    def __eq__(self, __value: object) -> bool:
        return super().__eq__(__value)
    
    def __hash__(self) -> int:
        return super().__hash__()

### Utility class takes care of generation function and heuristic function

In [11]:
class Utility:
    def __init__(self, size=3, use_ucs_strategy=True) -> None:
        self.size = size
        goal_state = np.roll(np.arange(self.size * self.size), -1)
        goal_state[-1] = 0
        self.goal_state = goal_state
        self.use_ucs_strategy = use_ucs_strategy

        self.label_column_wise_goal_state = {value: (
            index + 1) for index, value in enumerate(np.reshape(
            goal_state, (self.size, self.size)).T.flatten())} if not use_ucs_strategy else None

    def is_goal_state(self, board):
        return (board.board == self.goal_state).all()

    def get_successors(self, board) -> np.array:
        empty_tile = board.empty_tile

        successors = np.array([])

        if (isinstance(board, UCSBoard) and not isinstance(board, AStarBoard)):
            # Move up
            if empty_tile >= self.size:
                new_board = UCSBoard(board.board.copy(),
                                     empty_tile, board.path_cost + 1)
                new_board.board[empty_tile], new_board.board[empty_tile -
                                                             self.size] = new_board.board[empty_tile - self.size], new_board.board[empty_tile]
                new_board.empty_tile = empty_tile - self.size
                successors = np.append(successors, new_board)
            # Move down
            if empty_tile < self.size ** 2 - self.size:
                new_board = UCSBoard(board.board.copy(),
                                     empty_tile, board.path_cost + 1)
                new_board.board[empty_tile], new_board.board[empty_tile +
                                                             self.size] = new_board.board[empty_tile + self.size], new_board.board[empty_tile]
                new_board.empty_tile = empty_tile + self.size
                successors = np.append(successors, new_board)
            # Move left
            if empty_tile % self.size != 0:
                new_board = UCSBoard(board.board.copy(),
                                     empty_tile, board.path_cost + 1)
                new_board.board[empty_tile], new_board.board[empty_tile -
                                                             1] = new_board.board[empty_tile - 1], new_board.board[empty_tile]
                new_board.empty_tile = empty_tile - 1
                successors = np.append(successors, new_board)
            # Move right
            if empty_tile % self.size != self.size - 1:
                new_board = UCSBoard(board.board.copy(),
                                     empty_tile, board.path_cost + 1)
                new_board.board[empty_tile], new_board.board[empty_tile +
                                                             1] = new_board.board[empty_tile + 1], new_board.board[empty_tile]
                new_board.empty_tile = empty_tile + 1
                successors = np.append(successors, new_board)
        elif (isinstance(board, AStarBoard)):
            path_cost = board.path_cost
            inversion_count_horizontal = board.inversion_count_horizontal
            inversion_count_vertical = board.inversion_count_vertical
            empty_tile = board.empty_tile

            # move up, then only left-to-right top-to-bottom order is considered
            if (empty_tile >= self.size):
                new_board = AStarBoard(
                    board.board.copy(), empty_tile, path_cost + 1, inversion_count_horizontal, inversion_count_vertical, board.heuristic)
                # Swap the empty tile with the tile above it
                new_board.board[empty_tile], new_board.board[empty_tile -
                                                             self.size] = new_board.board[empty_tile - self.size], new_board.board[empty_tile]

                # Consider cols skipped tiles by empty_index to calculate inversion count
                # Traverse from empty_index - cols + 1 to empty_index - 1, if reach the tile that has value > successor[empty_index - cols],
                # then inv_count++, else inv_count--
                change_amount = sum(
                    1 if new_board.board[i] > new_board.board[empty_tile] else -1 for i in range(empty_tile - self.size + 1, empty_tile))
                # Update the inversion count and empty tile
                new_board.empty_tile = empty_tile - self.size
                new_board.inversion_count_horizontal += change_amount
                new_board.heuristic = self.hueristic_of(new_board)

                successors = np.append(successors, new_board)
            # move down, then only left-to-right top-to-bottom order is considered
            if (empty_tile < self.size ** 2 - self.size):
                new_board = AStarBoard(
                    board.board.copy(), empty_tile, path_cost + 1, inversion_count_horizontal, inversion_count_vertical, board.heuristic)
                # Swap the empty tile with the tile below it
                new_board.board[empty_tile], new_board.board[empty_tile +
                                                             self.size] = new_board.board[empty_tile + self.size], new_board.board[empty_tile]
                change_amount = sum(
                    1 if new_board.board[i] < new_board.board[empty_tile] else -1 for i in range(empty_tile + 1, empty_tile + self.size))
                # Update the inversion count and empty tile
                new_board.empty_tile = empty_tile + self.size
                new_board.inversion_count_horizontal += change_amount
                new_board.heuristic = self.hueristic_of(new_board)

                successors = np.append(successors, new_board)
            # move left, only top-to-bottom left-to-right order is considered
            if (empty_tile % self.size != 0):
                new_board = AStarBoard(
                    board.board.copy(), empty_tile, path_cost + 1, inversion_count_horizontal, inversion_count_vertical, board.heuristic)
                # Swap the empty tile with the tile on its left
                new_board.board[empty_tile], new_board.board[empty_tile -
                                                             1] = new_board.board[empty_tile - 1], new_board.board[empty_tile]
                # Reshape the board to a column-wise board
                column_wise_board = np.reshape(
                    new_board.board, (self.size, self.size)).T.flatten()
                # Convert the empty_tile to the column-wise board
                # start index of the empty tile in column-wise board
                start = ((empty_tile - 1) % self.size) * \
                    self.size + (empty_tile - 1) // self.size
                end = start + self.size  # index of the tile after swapping with the empty tile

                change_amount = sum(
                    1 if self.label_column_wise_goal_state[column_wise_board[i]] > self.label_column_wise_goal_state[column_wise_board[end]] else -1 for i in range(start + 1, end))

                # Update the inversion count and empty tile
                new_board.empty_tile = empty_tile - 1
                new_board.inversion_count_vertical += change_amount
                new_board.heuristic = self.hueristic_of(new_board)

                successors = np.append(successors, new_board)
            # move right, only top-to-bottom left-to-right order is considered
            if (empty_tile % self.size != self.size - 1):
                new_board = AStarBoard(
                    board.board.copy(), empty_tile, path_cost + 1, inversion_count_horizontal, inversion_count_vertical, board.heuristic)
                new_board.board[empty_tile], new_board.board[empty_tile +
                                                             1] = new_board.board[empty_tile + 1], new_board.board[empty_tile]
                column_wise_board = np.reshape(
                    new_board.board, (self.size, self.size)).T.flatten()
                start = (empty_tile % self.size) * \
                    self.size + empty_tile // self.size
                end = start + self.size

                change_amount = sum(
                    1 if self.label_column_wise_goal_state[column_wise_board[i]] < self.label_column_wise_goal_state[column_wise_board[start]] else -1 for i in range(start + 1, end))

                new_board.empty_tile = empty_tile + 1
                new_board.inversion_count_vertical += change_amount
                new_board.heuristic = self.hueristic_of(new_board)

                successors = np.append(successors, new_board)

        assert len(successors) > 0 and len(successors) <= 4

        return successors

    def inversion_count(self, board, axis=0) -> int:
        if axis == 0:
            # Count the number of inversions left-to-right and top-to-bottom order
            inversion_count_horizontal = sum(1 if (board[i] > board[j] and board[j] != 0 and board[i] != 0) else 0 for i in range(
                len(board)) for j in range(i + 1, len(board)))
            return inversion_count_horizontal

        assert axis == 1 and self.use_ucs_strategy == False and self.label_column_wise_goal_state is not None
        # Count the number of inversions top-to-bottom and left-to-right order
        # First, convert the board to a column-wise board
        column_wise_board = np.reshape(
            board, (self.size, self.size)).T.flatten()
        # Then count the number of inversions by:
        # 1. Traverse the column-wise board, if column_wise_board[j] != 0 and column_wise_board[i] != 0, and
        # label_column_wise_goal_state[column_wise_board[i]] > label_column_wise_goal_state[column_wise_board[j]], then inv_count++
        inversion_count_vertical = sum(1 if (column_wise_board[j] != 0 and column_wise_board[i] != 0 and self.label_column_wise_goal_state[column_wise_board[i]] > self.label_column_wise_goal_state[column_wise_board[j]]) else 0 for i in range(
            len(column_wise_board)) for j in range(i + 1, len(column_wise_board)))
        return inversion_count_vertical

    def hueristic_of(self, board=None, inv_count_hor=None, inv_count_ver=None) -> int:
        assert (inv_count_hor is not None and inv_count_ver is not None) or (
            inv_count_hor is None and inv_count_ver is None)
        if inv_count_hor is not None and inv_count_ver is not None:
            h_i_c = int(inv_count_hor / (self.size - 1)) + \
                inv_count_hor % (self.size - 1)
            v_i_c = int(inv_count_ver / (self.size - 1)) + \
                inv_count_ver % (self.size - 1)
            return h_i_c + v_i_c
        assert board is not None
        h_i_c = int(board.inversion_count_horizontal / (self.size - 1)) + \
            board.inversion_count_horizontal % (self.size - 1)
        v_i_c = int(board.inversion_count_vertical / (self.size - 1)) + \
            board.inversion_count_vertical % (self.size - 1)
        return h_i_c + v_i_c

    def generate_board(self, seed=None) -> UCSBoard:
        if seed is not None and seed > 0:
            np.random.seed(seed)
        # Generate a goal state for the board
        board = np.arange(self.size * self.size)

        # Shift the board to the left by 1, then fill the last tile with 0
        board = np.roll(board, -1)
        board[-1] = 0

        empty_tile = np.where(board == 0)[0][0]

        for _ in range(300):
            # Swap the empty tile with a random tile
            assert empty_tile >= 0 and empty_tile < self.size * self.size

            tile_to_swap = -1
            direction = np.random.randint(0, 4)
            if direction == 0 and empty_tile >= self.size:
                tile_to_swap = empty_tile - self.size
            elif direction == 1 and empty_tile < self.size * self.size - self.size:
                tile_to_swap = empty_tile + self.size
            elif direction == 2 and empty_tile % self.size != 0:
                tile_to_swap = empty_tile - 1
            elif direction == 3 and empty_tile % self.size != self.size - 1:
                tile_to_swap = empty_tile + 1
            if tile_to_swap != -1:
                board[empty_tile], board[tile_to_swap] = board[tile_to_swap], board[empty_tile]
                # Update the empty tile
                empty_tile = tile_to_swap
        if self.use_ucs_strategy:
            return UCSBoard(board, empty_tile, 0)
        inv_count_hor = self.inversion_count(board, axis=0)
        inv_count_ver = self.inversion_count(board, axis=1)
        return AStarBoard(board, empty_tile, 0, inv_count_hor, inv_count_ver, self.hueristic_of(inv_count_hor=inv_count_hor, inv_count_ver=inv_count_ver))

### We will need matplotlib to draw the board

In [12]:
class Plotter:
    @staticmethod
    def plot_board(board, title=""):
        reshape_board = np.reshape(
            board, (int(np.sqrt(len(board))), int(np.sqrt(len(board)))))

        print(title)
        print(reshape_board)

    @staticmethod
    def plot_path(parent_of, goal_state) -> None:
        path = []
        current = goal_state
        while current in parent_of:
            path.append(current)
            current = parent_of[current]

        print("Path length: ", len(path))
        if len(path) == 0:
            print("The initial board is the goal state")
            return
        if isinstance(path[0], UCSBoard) and not isinstance(path[0], AStarBoard):
            for i in range(len(path) - 1, -1, -1):
                Plotter.plot_board(
                    path[i].board, "Step " + str(len(path) - i) + "\nPath cost: " + str(path[i].path_cost))
        else:
            for i in range(len(path) - 1, -1, -1):
                Plotter.plot_board(path[i].board, "Step " + str(len(path) - i) + "\nHeuristic: " + str(
                    path[i].heuristic) + "\nPath cost: " + str(path[i].path_cost) + "\nTotal: " + str(path[i].get_total_cost()))

### Puzzle Solver class takes care of solving the puzzle

In [13]:
import heapq
import time
import os
import psutil


class PuzzleSolver:
    def __init__(self, customized_board=None, seed=-1, size=3, use_ucs_strategy=True) -> None:
        self.utility = Utility(size, use_ucs_strategy)
        if customized_board is not None:
            if use_ucs_strategy:
                board = UCSBoard(customized_board, np.where(
                    customized_board == 0)[0][0], 0)
            else:
                inv_count_hor = self.utility.inversion_count(
                    customized_board, axis=0)
                inv_count_ver = self.utility.inversion_count(
                    customized_board, axis=1)
                board = AStarBoard(customized_board, np.where(
                    customized_board == 0)[0][0], 0, inv_count_hor, inv_count_ver, self.utility.hueristic_of(inv_count_hor=inv_count_hor, inv_count_ver=inv_count_ver))
        self.customized_board = board if customized_board is not None else None

    def change_size(self, size):
        self.utility = Utility(size)

    def use_ucs_strategy(self):
        self.utility.use_ucs_strategy = True

    def use_a_star_strategy(self):
        self.utility.use_ucs_strategy = False

    def __ucs_algorithm(self) -> None:
        if not self.utility.use_ucs_strategy:
            self.use_ucs_strategy()
        initial_board = self.customized_board if self.customized_board is not None else self.utility.generate_board()
        Plotter.plot_board(initial_board.board, "Initial board")

        pq = []
        heapq.heapify(pq)
        # visited: dict of visited boards, frontier: dict of boards in the frontier
        visited, frontier = {}, {}
        parent_of = {}  # dict of parent of each board for path viewing

        frontier[initial_board] = initial_board.path_cost

        heapq.heappush(pq, initial_board)
        # Start timer
        start = time.time()
        while len(pq) > 0:
            current_board = heapq.heappop(pq)

            if current_board in visited:
                continue  # skip if the board is already visited, a compensation for not removing the board from the frontier

            if self.utility.is_goal_state(current_board):
                print("Found solution")

                end = time.time()
                print("Time elapsed: ", (end - start) * 1000, "ms")
                print("Total states expanded: ", len(visited))
                print("Total nodes generated: ", len(visited) + len(frontier))
                process = psutil.Process(os.getpid())
                print("Memory usage: ", process.memory_info().rss / 1024 ** 2, "MB")

                Plotter.plot_path(parent_of, current_board)
                break

            frontier.pop(current_board)
            visited[current_board] = True

            step_to_print = 50000 if len(current_board.board) == 9 else 500000 if len(
                current_board.board) == 16 else 5000000

            if len(visited) % step_to_print == 0:
                print("Total states expanded: ", len(visited))

            successors = self.utility.get_successors(current_board)

            for successor in successors:
                if successor not in visited and successor not in frontier:
                    heapq.heappush(pq, successor)
                    frontier[successor] = successor.path_cost
                    parent_of[successor] = current_board
                elif successor in frontier and successor.path_cost < frontier[successor]:
                    frontier[successor] = successor.path_cost
                    parent_of[successor] = current_board
                    heapq.heappush(pq, successor)

        if len(pq) == 0:
            print("Total states expanded: ", len(visited))
            print("Total nodes generated: ", len(visited) + len(frontier))
            print("No solution found")

            process = psutil.Process(os.getpid())
            print("Memory usage: ", process.memory_info().rss / 1024 ** 2, "MB")

    def __a_star_algorithm(self) -> None:
        if self.utility.use_ucs_strategy:
            self.use_a_star_strategy()
        initial_board = self.customized_board if self.customized_board is not None else self.utility.generate_board()
        assert isinstance(initial_board, AStarBoard)

        pq = []
        heapq.heapify(pq)
        # visited: dict of visited boards, frontier: dict of boards in the frontier
        visited, frontier = {}, {}
        parent_of = {}  # dict of parent of each board for path viewing

        frontier[initial_board] = initial_board.get_total_cost()

        heapq.heappush(pq, initial_board)
        # Start timer
        start = time.time()
        while len(pq) > 0:
            current_board = heapq.heappop(pq)

            if current_board in visited:
                continue  # skip if the board is already visited, a compensation for not removing the board from the frontier

            if self.utility.is_goal_state(current_board):
                print("Found solution")
                end = time.time()
                print("Time elapsed: ", (end - start) * 1000, "ms")
                print("Total states expanded: ", len(visited))
                print("Total nodes generated: ", len(visited) + len(frontier))
                process = psutil.Process(os.getpid())
                print("Memory usage: ", process.memory_info().rss / 1024 ** 2, "MB")

                Plotter.plot_path(parent_of, current_board)
                break

            frontier.pop(current_board)
            visited[current_board] = True

            step_to_print = 50000 if len(current_board.board) == 9 else 500000 if len(
                current_board.board) == 16 else 10000000

            if len(visited) % step_to_print == 0:
                print("Total states expanded: ", len(visited))

            successors = self.utility.get_successors(current_board)

            for successor in successors:
                if successor not in visited and successor not in frontier:
                    heapq.heappush(pq, successor)
                    frontier[successor] = successor.get_total_cost()
                    parent_of[successor] = current_board
                elif successor in frontier and successor.get_total_cost() < frontier[successor]:
                    frontier[successor] = successor.get_total_cost()
                    parent_of[successor] = current_board
                    heapq.heappush(pq, successor)
        if len(pq) == 0:
            print("Total states expanded: ", len(visited))
            print("Total nodes generated: ", len(visited) + len(frontier))
            print("No solution found")

            process = psutil.Process(os.getpid())
            print("Memory usage: ", process.memory_info().rss / 1024 ** 2, "MB")

    def solve(self) -> None:
        if self.utility.use_ucs_strategy:
            self.__ucs_algorithm()
        else:
            self.__a_star_algorithm()

### Main function

In [14]:
if __name__ == "__main__":
    while True:
        strategy = int(input("N-puzzle solver\n1. Use UCS strategy\n2. Use A* strategy\n:"))
        assert strategy == 1 or strategy == 2
        board_size = int(input("Enter the size of the board (8, 15, 35): "))
        assert board_size == 8 or board_size == 15 or board_size == 35
        size = 3 if board_size == 8 else 4 if board_size == 15 else 6
        # Ask for customized board
        customized_board = None

        customized_board_str = input(
            "Do you want to use a customized board? (y/n): ")
        if customized_board_str == "y":
            customized_board = np.array(
                list(map(int, input("Enter the board: ").split())))
            assert len(customized_board) == size * size
        if strategy == 1:
            solver = PuzzleSolver(size=size, use_ucs_strategy=True, customized_board=customized_board)
        else:
            solver = PuzzleSolver(size=size, use_ucs_strategy=False, customized_board=customized_board)
        solver.solve()
        print("Press enter to continue")
        input()
    
    # ###############################################################################
    # ## Easiest board
    # solver = PuzzleSolver(size=3, use_ucs_strategy=True, customized_board=np.array([1, 2, 3, 4, 5, 6, 7, 0, 8]))
    # solver.solve()
    # ## Hardest board
    # solver = PuzzleSolver(size=3, use_ucs_strategy=True, customized_board=np.array([8, 6, 7, 2, 5, 4, 3, 0, 1]))
    # solver.solve()
    # ###############################################################################
    # ## Easiest board
    # solver = PuzzleSolver(size=4, use_ucs_strategy=False, customized_board=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11 ,12, 13, 14, 0, 15]))
    # solver.solve()
    # ## Hardest board
    # solver = PuzzleSolver(size=4, use_ucs_strategy=False, customized_board=np.array([15, 14, 13, 12, 11, 10, 9, 8, 7, 6 ,5, 4, 3, 2, 1, 0]))

Found solution
Time elapsed:  0.0 ms
Total states expanded:  0
Total nodes generated:  1
Memory usage:  89.80859375 MB
Path length:  0
The initial board is the goal state
Total states expanded:  0
Total nodes generated:  1
No solution found
Memory usage:  89.80859375 MB
Press enter to continue


ValueError: invalid literal for int() with base 10: '8 7 6 5 4 3 2 1 0'